#### Challenge 47: Bleichenbacher's PKCS 1.5 Padding Oracle (Simple Case)

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import math
import base64
import cryptopals as cp

from decimal import *

<div class="alert alert-block alert-info">   
    
<div class="alert alert-block alert-warning">
    
#### **Degree of difficulty: Moderate**
These next two challenges are the hardest in the entire set.

</div>
    
Let us Google this for you: [Chosen ciphertext attacks against protocols based on the RSA encryption standard](http://lmgtfy.com/?q=%22Chosen+ciphertext+attacks+against+protocols+based+on+the+RSA+encryption+standard%22)

This is Bleichenbacher from CRYPTO '98; I get a bunch of .ps versions on the first search page.

Read the paper. It describes a padding oracle attack on PKCS#1v1.5. The attack is similar in spirit to the CBC padding oracle you built earlier; it's an "adaptive chosen ciphertext attack", which means you start with a valid ciphertext and repeatedly corrupt it, bouncing the adulterated ciphertexts off the target to learn things about the original.

This is a common flaw even in modern cryptosystems that use RSA.

It's also the most fun you can have building a crypto attack. It involves 9th grade math, but also has you implementing an algorithm that is complex on par with finding a minimum cost spanning tree.

The setup:

- Build an oracle function, just like you did in the last exercise, but have it check for plaintext[0] == 0 and plaintext[1] == 2.
- Generate a 256 bit keypair (that is, p and q will each be 128 bit primes), [n, e, d].
- Plug d and n into your oracle function.
- PKCS1.5-pad a short message, like "kick it, CC", and call it "m". Encrypt to to get "c".
- Decrypt "c" using your padding oracle.

For this challenge, we've used an untenably small RSA modulus (you could factor this keypair instantly). That's because this exercise targets a specific step in the Bleichenbacher paper --- Step 2c, which implements a fast, nearly O(log n) search for the plaintext.

Things you want to keep in mind as you read the paper:

- RSA ciphertexts are just numbers.
- RSA is "homomorphic" with respect to multiplication, which means you can multiply c * RSA(2) to get a c' that will decrypt to plaintext * 2. This is mindbending but easy to see if you play with it in code --- try multiplying ciphertexts with the RSA encryptions of numbers so you know you grok it.
- What you need to grok for this challenge is that Bleichenbacher uses multiplication on ciphertexts the way the CBC oracle uses XORs of random blocks.
- A PKCS#1v1.5 conformant plaintext, one that starts with 00:02, must be a number between 02:00:00...00 and 02:FF:FF..FF --- in other words, 2B and 3B-1, where B is the bit size of the modulus minus the first 16 bits. When you see 2B and 3B, that's the idea the paper is playing with.

To decrypt "c", you'll need Step 2a from the paper (the search for the first "s" that, when encrypted and multiplied with the ciphertext, produces a conformant plaintext), Step 2c, the fast O(log n) search, and Step 3.

Your Step 3 code is probably not going to need to handle multiple ranges.

We recommend you just use the raw math from paper (check, check, double check your translation to code) and not spend too much time trying to grok how the math works.

</div>    

---

<div class="alert alert-block alert-info">   

- Generate a 256 bit keypair (that is, p and q will each be 128 bit primes), [n, e, d].
    
</div> 

In [2]:
valid_params = False

while not(valid_params):
    
    print('.', end='')
    p = number.getPrime(256 // 2)
    q = number.getPrime(256 // 2)

    n = (p * q)

    et = (p-1) * (q-1)
    e = 3

    d = cp.invmod(e, et)

    # Check parameters:
    PT = random.randint(0, 2**32-1)
    valid_params = (pow(pow(PT, e, n), d, n) == PT)

print(f"\nGenerated working parameters:\n")
print(f"e={e}\nd={d}\nn={n}")

...
Generated working parameters:

e=3
d=29101896161579898797904737493153263735320159038484732736618355837704120871851
n=43652844242369848196857106239729895603398343547168582480221614873868167538779


In [3]:
# Set decimal precision to handle math  for this challenge...
getcontext().prec = int(math.log2(n))

In [4]:
# e, d, n = cp.genRSA_keypair(1024)

<div class="alert alert-block alert-info">   
    
- Build an oracle function, just like you did in the last exercise, but have it check for plaintext[0] == 0 and plaintext[1] == 2.
- Plug d and n into your oracle function.
- PKCS1.5-pad a short message, like "kick it, CC", and call it "m". Encrypt to to get "c".

</div> 

Here is [RFC2313](https://tools.ietf.org/html/rfc2313) that describes PKCS1.5 padding for RSA.  For encryption, we assume block type 02.  

The encryption block `EB` is constructed as:

`EB = 00 || BT || PS || 00 || D`

where:

- `BT = 02`
- `PS` is the padding string, which is pseudorandomly generated w/ no `00`'s
- `D` is the data

The length of the encryption block `EB` is equal to `k`, where `k` is the length of the modulus in octets (bytes).  

In [5]:
def remove_pkcs15_padding(byte_data, n):
    
    k = math.ceil(math.log2(n)) // 8
    
    if len(byte_data)==(k-1):        
        byte_data = b'\x00' + byte_data
    
    if not(len(byte_data) == k):
        return(False)

    if not(byte_data[1] == 0x02):
        return(False)
    
    data_idx = byte_data.find(b'\x00', 2) + 1   
    payload = byte_data[data_idx:]
    
    return(payload)
    
def validate_pkcs15_padding(byte_data, n):

    k = math.ceil(math.log2(n)) // 8
    
    if len(byte_data)==(k-1):        
        byte_data = b'\x00' + byte_data
    
    if not(len(byte_data) == k):
        return(False)

    if not(byte_data[1] == 0x02):
        return(False)
    
    data_idx = byte_data.find(b'\x00', 3) + 1
    
    return not(data_idx == 0)

def simple_validate_padding(byte_data, n):

    k = math.ceil(math.log2(n)) // 8
    
    if len(byte_data)==(k-1):        
        byte_data = b'\x00' + byte_data
    
    if not(len(byte_data) == k):
        return(False)

    if not(byte_data[1] == 0x02):
        return(False)

    return(True)

def pkcs15_pad(data, n):
    
    k = math.ceil(math.log2(n)) // 8
    data_len = len(data)
    ps_len = k - data_len - 3
    
    b00 = b'\x00'
    BT = b'\x02'
    PS = []
    
    for ii in range(ps_len):
        PS.append(random.randint(1, 255))
    
    EB = b00 + BT + bytes(PS) + b00 + data
    
    return(EB)

def bytes_to_bigint(byte_data):
    
    return(int(byte_data.hex(), 16))

def bigint_to_bytes(int_data):
    
    hex_data = hex(int_data)[2:]
    if len(hex_data) % 2:
        hex_data = '0' + hex_data
    return(bytes.fromhex(hex_data))


In [6]:
def challenge47_oracle(ciphertext):
    
    plaintext = pow(ciphertext, d, n)
    plaintext_hex = hex(plaintext)[2:]
    if (len(plaintext_hex) % 2):
        plaintext_hex = '0' + plaintext_hex
    plaintext_bytes = bytes.fromhex(plaintext_hex)
    
    return( simple_validate_padding(plaintext_bytes, n) )
        

In [7]:
x = pkcs15_pad(b'help', n)
x_int = bytes_to_bigint(x)
bigint_to_bytes(x_int)

b'\x02\xe4h.{\xa8?\xb2%\xd7k\xe4\xc2\xf8\x1f@X\x8a{\xf1mA\x1d\x8a\xe1\x00help'

In [8]:
# Test everything out to make sure it works....
m = bytes_to_bigint(pkcs15_pad(b'This is a test message', n))
c = pow(m, e, n)
m_r = bigint_to_bytes(pow(c, d, n))
assert(validate_pkcs15_padding(m_r, n))
m_r = remove_pkcs15_padding(m_r, n)
print(m_r.decode())

This is a test message


In [9]:
m = bytes_to_bigint(pkcs15_pad(b'kick it, cc', n))
c = pow(m, e, n)

<div class="alert alert-block alert-info">   

- Decrypt "c" using your padding oracle.
    
</div>

In [10]:
# Implement Step #3 - Narrowing set of solutions.

def update_intervals(M_Last, s):
    
    M = []

    for interval in M_Last:

        a, b = interval[1], interval[2]
        r_min = math.floor((a*s - 3*B + 1) / n) 
        r_max = math.ceil((b*s - 2*B) / n) 
        
        for r in range(r_min, r_max+1):
            new_a = max(a, math.ceil((2*B+r*n)/s)) 
            new_b = min(b, math.floor((3*B - 1 + r*n) / s)) 
            if new_b >= new_a:
                M.append([r, new_a, new_b])
    
    if M == []:
        return(M_Last)
        print('Houston, we have a problem')
    else:
        return(M)

In [11]:
# Implement Step 2.a - find an initial s that results in conforming PKCS padding:

k = math.ceil(math.log2(n)) // 8
B = Decimal(2**(8*(k-2)))
s0 = math.ceil(n / (3*B))

conforming = False
while not(conforming):
    
    c_ = c*(pow(s0, e, n)) % n
    conforming = challenge47_oracle(c_)
    if not(conforming):
        s0 += 1    

In [ ]:
s = s0

M = [[0, 2*B, 3*B - 1]]
M = update_intervals(M, s)

a = Decimal(0)
b = Decimal(0)
r = Decimal(0)

done = False
count = 0
while not(done):

    conforming = False
    print(M)
    if len(M) > 1:
        
        # Step 2.b:  Searching with more than one interval left
        uh_oh = s-1
        while not(conforming):
            if s == uh_oh:
                print('CRAP!')
            count += 1
            if count % 10000 == 0:
                print('x', end='')
            s = (s + 1) % n
            c_ = c*(pow(s, e, n)) % n
            conforming = challenge47_oracle(c_)
            
    else: 
        
        # Step 2.c:  Searching with one interval left
        
        a, b = M[0][1], M[0][2]        
        r = math.floor(2*((b*s - 2*B) / n)) 
        print('.', end='')
        while not(conforming):
            
            s_min = math.floor((2*B + r*n) / b) 
            s_max = math.ceil((3*B + r*n) / a) 
            s = s_min

            while not(conforming) and (s != (s_max + 1)):
                
                c_ = c*(pow(s, e, n)) % n
                conforming = challenge47_oracle(c_)
                if not(conforming):
                    s += 1
                
            r = (r + 1) % n
                
    M = update_intervals(M, s)
    if (len(M) == 1) and (M[0][1] == M[0][2]): 
        done = True

[[1, 20603935705704694264213746342859903428595752444881131250038809892128640, 20603938963295247371954082684102865876204508355963374591078197489099668]]
.[[5, 20603936328534173878085053550047232807128735159224373678059894185687772, 20603936980052469007704273973881582702777708560225680648984756144779964]]
.[[10, 20603936649511896737521430934221213732439679848152691656534387902786256, 20603936975271059678008717313752509321083410051838425886022479068602971]]
.[[20, 20603936810000769529737406597481825800985261259313843797296562253107512, 20603936972880354843900740975924948476382553315034042449971555510360497]]
.[[40, 20603936890245208766470075805141743768060202229318240256074879707337614, 20603936971685002384531699809882226803517413885434285821631214299586894]]
.[[80, 20603936930367429094992610082136227155485038605855858779047791040714638, 20603936971087326144268415540680068837816346604164186198252929823126940]]
.[[24707, 206039369421771472794265770357045903469548653768332158410454407209446

In [ ]:
recovered_msg = remove_pkcs15_padding(bigint_to_bytes(M[0][1]), n)
print(recovered_msg)

In [ ]:
s = s0

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)